<a href="https://colab.research.google.com/github/CHINNA777UMA/23B21A4296_FMML/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Ans.
Yes, averaging the validation accuracy across multiple splits generally provides more consistent and reliable results. Here's why:

1. **Reduced Variance**: By using multiple splits (often referred to as k-fold cross-validation), you minimize the impact of any single split's peculiarities or biases, which can occur due to random variations in the data.

2. **Better Generalization**: Averaging over several splits gives you a better estimate of the model's performance on unseen data. It reflects how well the model generalizes across different subsets of the dataset.

3. **Detection of Overfitting**: Consistency in validation accuracy across splits can help identify whether a model is overfitting to the training data. If the accuracy varies widely, it may indicate that the model is sensitive to the specific data it was trained on.

4. **Robustness**: This approach is more robust to outliers and anomalies in the data, as the effects of any individual instance are averaged out over multiple evaluations.

Overall, while averaging validation accuracy can lead to a more stable estimate of model performance, it's essential to ensure that the splits are representative of the entire dataset to maintain the validity of the results.
2. Does it give more accurate estimate of test accuracy?
Ans
Averaging validation accuracy across multiple splits does not directly provide a more accurate estimate of test accuracy, but it does lead to a more reliable and stable estimate of model performance. Here’s how it works:

1. **Bias-Variance Tradeoff**: Averaging validation scores helps mitigate variance in model evaluation. While it doesn’t change the bias of the model, it provides a clearer view of how well the model performs across different subsets of the data.

2. **Model Selection**: By using k-fold cross-validation (or other splitting methods), you can select the best model or hyperparameters based on averaged validation performance. This selection can indirectly lead to better test accuracy since you’re choosing the model that generalizes well.

3. **Estimating Generalization Error**: The average validation accuracy gives a more comprehensive picture of how the model might perform on unseen data. However, this still might not perfectly correlate with the actual test accuracy due to the inherent differences between the validation set used during training and the final test set.

4. **Independent Test Set**: The true measure of a model’s performance is how it performs on a completely independent test set. The average validation accuracy helps inform model tuning and selection, but the ultimate test accuracy will depend on factors like data distribution and the model's generalization capabilities.

In summary, while averaging validation accuracy provides a more reliable estimate of a model's ability to generalize, the only definitive measure of test accuracy is obtained from evaluating the model on an unseen test set.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Ans
The number of iterations in the context of model evaluation, such as during k-fold cross-validation or other iterative processes, can significantly impact the quality of the estimate for model performance. Here's how the number of iterations affects the estimate:

1. **Reduced Variance**: Increasing the number of iterations or folds (e.g., using 10-fold instead of 5-fold cross-validation) generally leads to a more stable estimate of model performance. Each iteration evaluates the model on different subsets of the data, which helps average out any random fluctuations and provides a more consistent accuracy measure.

2. **Improved Reliability**: More iterations can improve the reliability of the estimate by providing a larger number of performance evaluations. This helps ensure that the estimate is less influenced by any particular split of the data, leading to a more accurate reflection of the model's generalization capability.

3. **Convergence**: In some algorithms, especially those involving stochastic optimization (like stochastic gradient descent), more iterations can lead to better convergence towards an optimal solution. This can enhance the model's performance, but it's important to monitor for overfitting, especially if the training process is not properly regularized.

4. **Diminishing Returns**: While increasing the number of iterations can yield better estimates, there are diminishing returns. Beyond a certain point, the additional iterations may contribute less to improving the estimate's accuracy, and computational costs may rise without significant benefits.

5. **Trade-off with Computation**: More iterations or folds will lead to longer computation times. Therefore, it's essential to balance the desire for a better estimate with available resources and time constraints.

In summary, while higher iterations generally lead to better and more reliable estimates of model performance, the improvements may diminish after a certain point, and the computational cost should be taken into account. A common practice is to choose a moderate number of iterations (like 5 or 10 folds in cross-validation) that balances accuracy and computational efficiency.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Ans
Increasing the number of iterations (or folds in k-fold cross-validation) can help mitigate some challenges associated with a very small training or validation dataset, but it doesn't entirely solve the fundamental issues. Here are some considerations:

### Advantages of Increasing Iterations

1. **Better Utilization of Data**: With k-fold cross-validation, more iterations allow for each data point to be used for both training and validation multiple times. This can help you make the most of a limited dataset.

2. **Variance Reduction**: More folds can reduce the variance in performance estimates, leading to a more stable and reliable evaluation of the model. Each fold provides a different perspective on how the model performs, helping to average out the noise.

3. **Model Selection**: With multiple iterations, you can better assess which model or hyperparameters work best, even if the dataset is small. This can lead to improved generalization when you apply the model to unseen data.

### Limitations of Increasing Iterations

1. **Insufficient Data**: While increasing the number of folds can help, it doesn’t increase the amount of actual training data. Small datasets may still lead to overfitting, as the model might learn noise or specific patterns in the limited training data.

2. **Validation Set Size**: Each fold in k-fold cross-validation uses a portion of the data for validation. If the dataset is too small, the validation sets might not be representative of the overall data distribution, leading to unreliable validation scores.

3. **Bias in Estimates**: With very small datasets, even multiple iterations may not provide a true estimate of model performance. The model may perform well on the training data and poorly on unseen data due to the limited diversity in the dataset.

4. **Computational Overhead**: Increasing iterations means more computations, which may not be efficient if the dataset is already small.

### Alternative Strategies

1. **Data Augmentation**: If possible, use data augmentation techniques to artificially increase the size of your training dataset. This can help the model learn more robust features.

2. **Transfer Learning**: Consider using pre-trained models and fine-tuning them on your small dataset, which can lead to better performance than training a model from scratch.

3. **Bootstrapping**: Bootstrapping can provide a way to create multiple training sets from your small dataset, allowing you to better estimate model performance.

4. **Ensemble Methods**: Utilizing ensemble methods can help improve robustness, as they can combine predictions from multiple models trained on slightly different data distributions.

### Conclusion

While increasing the number of iterations in cross-validation can improve the evaluation of model performance on small datasets, it does not completely overcome the limitations posed by the lack of sufficient data. Combining this approach with other techniques, such as data augmentation or transfer learning, may yield better results when working with small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.